In [2]:
import pandas as pd
import numpy as np 
import time
import os

## Notebook to test and compare different prompts

In [3]:

def upload_analyze_papers(food, hazard, pdfs, API_Key, question='default'):
    all_results = pd.DataFrame(columns=['filename', 'foodname','species','othername', 'hazard', 'quote', 'location', 'pos/neg', 'how','howquote']) 

    
    # Load papers into chatpdf
    # dummy loop to only upload 12/minute
    uploads = 0
    start_time = time.time()

    for file in pdfs:
        # Check if we've reached the upload limit
        if uploads >= 12:
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                # If uploads exceed the limit in less than a minute, wait for the remaining time
                wait_time = 60 - elapsed_time
                print(f'Upload limit reached. Waiting for {wait_time:.2f} seconds...')
                time.sleep(wait_time)
            uploads = 0  # Reset the counter and start a new minute
            start_time = time.time()

        print(f'loading file "{os.path.basename(file)}" ')
        files = [('file', ('file', open(file, 'rb'), 'application/octet-stream'))]
        headers = {'x-api-key': API_Key}
        response = requests.post('https://api.chatpdf.com/v1/sources/add-file', headers=headers, files=files)
        
        # Update the upload counter
        uploads += 1

        if response.status_code == 200:
            sID = response.json()['sourceId']
        else:
            print('Status:', response.status_code)
            print('Error:', response.text)
            continue

        # 'Read' papers in chatpdf
        headers = {
            'x-api-key': API_Key,
            "Content-Type": "application/json"
        }
        print('analyzing file...')
        if question == 'default':
            question = f'1a) provide a quote from the text about how {hazard} impacts {food}? 2a) in what region is this study 3a) does {hazard} negatively or positively impact {food} (reply only negatvie/positive) 4a) would you classify the impact on {food} as direct physiology, indirect abiotic, or indirect biotic?  5) can you give a quote providing an example of this?'

        data = {
            'sourceId': sID,
            'messages': [
                {
                    'role': "user",
                    'content': question,
                }
            ]
        }
        response = requests.post(
            'https://api.chatpdf.com/v1/chats/message', headers=headers, json=data)

        if response.status_code == 200:
            print('result:', response.json()['content'])
            results = response.json()['content']
            segments = results.split('a)')  # Split the text using regex
            if len(segments) >= 5:
                quote = segments[1].strip()
                location = segments[2].strip()
                posneg = segments[3].strip()
                how = segments[4].strip()
                howquote = segments[5].strip()

                # Create a row dictionary for this PDF
                row_data = {
                    'filename': os.path.basename(file),
                    list: food,
                    'hazard': hazard,
                    'quote': quote,
                    'location': location,
                    'pos/neg': posneg,
                    'how': how,
                    'howquote': howquote
                }

                # Append the row data to the results
                all_results = pd.concat([all_results, pd.DataFrame([row_data])], ignore_index=True)
        else:
            print('Status:', response.status_code)
            print('Error:', response.text)
        print("\n")

    return all_results

In [ ]:


foods = ['Bay leaf','Bignose unicornfish','Button mushroom','Centropristis philadelphica','Chinese softshell turtle','Corica soborna','Ganges river sprat','Grasshopper','Large yellow croaker','Larimichthys croceus','Leucoraja erinacea','Little skate','mealworm','Naso vlamingii','Orange-spotted grouper','Ordways brotula','Pusa hispida','Ringed seal','Rock sea bass','Tenebrio molitor','Trionyx sinensis','Triticum aestivum','wheat']
hazards = ['atmospheric CO2 increases','ocean acidification','ozone','storms','warming','precipitation','precipitation','fires','warming','warming','ocean acidification','ocean acidification','warming','ocean acidification','natural cover change','floods','heatwaves','Heatwaves','storms','warming','warming','drought','drought']
API_Key = 'sec_6LIDxgLBHBqmhkVam818PYYXqervcPSX' #make a user and get api key from https://www.chatpdf.com/docs/api/backend

question  = 'XX'




To do for prompt test
- load papers
- get comporable output as human df
- make list/dict/df with question and dfID


To do for big tool
- find effective way to combine bibtxt csv and/or pdfs into groupings that make sense
- either append resutls here or on pypaperbot csv
- subset run to 'smaller' batches 
